In [18]:
##### importing custom modules from the projects folder
import sys
from pathlib import Path
# Add project root to sys.path
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.append(str(PROJECT_ROOT))
import modules.scrapers as scrapers
##### -------------------------------------------------
from datetime import date

##### PARAMS ######
league = 'nfl'
export = True
today = date.today()
season = 2025
inseason = False
week = int(input("What week is it >>>>>> ? "))
print(today, "Week:", week)

scraper = scrapers.scrapers(
    season = season,
    week = week,
    today = today
)

2025-08-17 Week: 0


### ONLY RUNS INSEASON FOR UPDATING ACTUAL GAME RESULTS AT THE END OF THE WEEK

In [ ]:

#scraper.cbs_game_scores(inseason = inseason, export = export)

### can run these proj and rank scrapes offseason or inseason

In [199]:
# in- and off-season
scraper.cbs_projections(inseason = inseason, export = export)

(441, 60)

In [ ]:
# doesn't work anymore
#scraper.cbs_rankings(export = export)

In [ ]:
# in- and off-season
scraper.ffp_ecr_rankings(inseason = inseason, export = export)

In [198]:
# in- and off-season
scraper.espn_projections(inseason = inseason, export = export)

(600, 60)

In [ ]:
# in- and off-season
scraper.espn_rankings(inseason = inseason, export = export)

In [195]:
# in- and off-season
scraper.nfl_projections(inseason = inseason, export = export)

<string>:245: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


(1100, 60)

In [ ]:
# in- and off-season
scraper.nfl_rankings(inseason = inseason, export = export)

In [ ]:
scraper.bp_lines(export = export)

#  ADP

In [95]:
scraper.cbs_adp(
    export = export
)

In [ ]:
scraper.ffp_adp(
        export = export
    )

In [ ]:
scraper.espn_adp(
        export = export
    )

# Process

In [38]:
scraper.generate_id_maps()
scraper.process_projections()
scraper.process_rankings()

query successful
query successful
query successful
query successful
query successful
query successful


<string>:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<string>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


1883 missing players..
query successful


<string>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


query successful
successfully added data to player
inserts completed..
cbs
Empty DataFrame
Columns: [playerId, cbsId, cbsName]
Index: []
(sqlalchemy.exc.InvalidRequestError) A value is required for bind parameter 'cbsId'
[SQL: UPDATE player SET cbsId = %(cbsId)s, cbsName = %(cbsName)s WHERE playerId = %(playerId)s]
(Background on this error at: https://sqlalche.me/e/20/cd3x)


<string>:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<string>:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


7 missing players..
query successful


<string>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


query successful
successfully added data to player
inserts completed..
fantasyPros
nfl
updates completed...


,outlet,date,season,week,group,expert,rank,high,low,playerId
0,1,2022-08-22,2022,0,QB,1,1.0,None,None,88
1,1,2022-08-22,2022,0,QB,2,1.0,None,None,88
2,1,2022-08-22,2022,0,QB,3,1.0,None,None,88
3,1,2022-08-22,2022,0,QB,4,1.0,None,None,88
4,1,2022-08-22,2022,0,QB,5,1.0,None,None,88
...,...,...,...,...,...,...,...,...,...,...
379,4,2025-08-16,2025,0,DB,13,26.0,None,None,15327
380,4,2025-08-16,2025,0,DB,13,27.0,None,None,1332
381,4,2025-08-16,2025,0,DB,13,28.0,None,None,1754
382,4,2025-08-16,2025,0,DB,13,29.0,None,None,5784


In [139]:
# all 2024 is loaded
#scraper.process_game_lines(bettingTableName = 'betting')

In [35]:

scraper.process_rankings()


<string>:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
<string>:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


13 missing players..
query successful


<string>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


query successful
successfully added data to player
inserts completed..
espn
fantasyPros
nfl
updates completed...


,outlet,date,season,week,group,expert,rank,high,low,playerId
0,1,2022-08-22,2022,0,QB,1,1.0,None,None,1204
1,1,2022-08-22,2022,0,QB,2,1.0,None,None,1204
2,1,2022-08-22,2022,0,QB,3,1.0,None,None,1204
3,1,2022-08-22,2022,0,QB,4,1.0,None,None,1204
4,1,2022-08-22,2022,0,QB,5,1.0,None,None,1204
...,...,...,...,...,...,...,...,...,...,...
379,4,2025-08-16,2025,0,DB,13,26.0,None,None,15327
380,4,2025-08-16,2025,0,DB,13,27.0,None,None,1332
381,4,2025-08-16,2025,0,DB,13,28.0,None,None,1754
382,4,2025-08-16,2025,0,DB,13,29.0,None,None,5784


(135021, 10)